In [1]:
import pandas as pd
import random
from datetime import datetime
import time
from itertools import islice

In [2]:
def unique(sequence): #removes duplicates from a list whilst preserving order
    visited = set()
    return [x for x in sequence if not (x in visited or visited.add(x))]

In [3]:
url = "output_datasets/station_times_for_fixing_data_nextbike_2020-07.csv.gz"

In [4]:
df = pd.read_csv(url, sep=';', low_memory=False)

In [5]:
fmt = '%Y-%m-%d %H:%M:%S'

In [6]:
df.columns

Index(['lat', 'lng', 'time_begin', 'time_end', 'standing_time_minutes', 'uid',
       'name', 'address', 'bike', 'bike_racks', 'bikes', 'booked_bikes',
       'free_racks', 'free_special_racks', 'maintenance', 'number',
       'place_type', 'rack_locks', 'special_racks', 'spot', 'terminal_type',
       'city', 'country_code', 'company', 'timezone', 'available_bikes'],
      dtype='object')

In [7]:
pd.set_option('display.max_columns()', None)

In [8]:
new_df = pd.DataFrame(columns= ['old_index', 'lat', 'lng', 'time_begin', 'time_end', 'standing_time_minutes', 'uid', 'name', 'address',
                                'bike', 'bike_racks', 'bikes', 
                                'booked_bikes', 'free_racks', 'free_special_racks', 'maintenance', 'number', 
                                'place_type', 'rack_locks', 'special_racks', 'spot', 'terminal_type', 'city',
                                'country_code', 'company', 'timezone', 'available_bikes'] ) 
#empty dataframe with the new column structure


#### OLD_INDEX is gonna be used later when we itterate trough smaller data frames containing only data rows with same uid. 
#### These subset data frames create new indexes but we use old index to compare positions based on the original place in the main df

In [9]:
df

,lat,lng,time_begin,time_end,standing_time_minutes,uid,name,address,bike,bike_racks,bikes,booked_bikes,free_racks,free_special_racks,maintenance,number,place_type,rack_locks,special_racks,spot,terminal_type,city,country_code,company,timezone,available_bikes
0,50.944121,7.023142,2020-07-01 00:00:01.668676,2020-07-03 18:42:02.297568,4002.0,31797250,BIKE 22822,NaN,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin,1093
1,50.957413,7.011876,2020-07-03 18:54:01.772866,2020-07-03 18:54:01.772866,0.0,31952802,BIKE 22822,NaN,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin,1136
2,50.938503,6.937919,2020-07-03 22:51:02.841407,2020-07-03 22:51:02.841407,0.0,31966763,BIKE 22822,NaN,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin,1099
3,50.938537,6.937958,2020-07-03 22:52:02.436098,2020-07-04 02:01:02.253767,189.0,31966763,BIKE 22822,NaN,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin,1097
4,50.977123,6.963487,2020-07-04 02:25:02.497300,2020-07-04 02:25:02.497300,0.0,31975314,BIKE 22822,NaN,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin,1103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118881,50.949828,6.991957,2020-07-25 23:10:02.628905,2020-07-25 23:57:03.120293,47.0,33155146,BIKE 22910,NaN,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin,1062
118882,50.950848,6.980691,2020-07-26 00:06:02.300512,2020-07-26 00:06:02.300512,0.0,33157584,BIKE 22910,NaN,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin,1056
118883,50.950854,6.980732,2020-07-26 00:07:02.800269,2020-07-26 01:05:03.062888,58.0,33157584,BIKE 22910,NaN,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin,1055
118884,50.936356,7.002997,2020-07-26 02:49:02.183598,2020-07-26 02:49:02.183598,0.0,33161266,BIKE 22910,NaN,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin,1060


In [10]:
uidsToList = df['uid'].tolist()
uidsList = unique(uidsToList) #removes duplicates from a list whilst preserving order

In [11]:
len(uidsList)

69238

In [12]:
df.reset_index(inplace=True)

In [13]:
df.rename(columns={"index": "old_index"}, inplace=True) #create duplicate column of indexes - old_index

### We want to merge all the rows with the same uid into one row
### First we need to check GPS errors

Old index order is used to interupt same uid processing if two rows are not in a sequence in original data frame. This eliminates the error that was created in cases when one uid appeared for to riddes that are sepparated by many other rides on the same bike. This error is related to how the uid is generated

In [14]:
questionable_changes = {} #creating dictionary to hold uid:difference 
for uid in uidsList: 
    single_uid_records = df[df['uid']==uid]
    first_row_data = single_uid_records.iloc[0]
    last_end_time = first_row_data['time_end']
    last_oi = first_row_data['old_index']

    for index, row in islice(single_uid_records.iterrows(), 1, None):
        if last_oi+1 != df['old_index'][index]: #using old index to interupt same uid processing if two rows are not in a sequence in original data frame
            break
        t_begin = df['time_begin'][index].split('.')[0]
        t_end_previous = last_end_time.split('.')[0]    

        d1 = datetime.strptime(t_begin, fmt)
        d2 = datetime.strptime(t_end_previous, fmt)

        d1_ts = time.mktime(d1.timetuple())
        d2_ts = time.mktime(d2.timetuple())

        minsDiff = round((int(d1_ts-d2_ts) / 60), 0)

        last_end_time = row['time_end']
        last_oi = df['old_index'][index]
        if minsDiff > 1:
            questionable_changes[uid] = minsDiff
        


In [15]:
questionable_changes

{32905555: 4.0,
 32952412: 138.0,
 32409885: 4.0,
 32810528: 94.0,
 32381189: 2.0,
 32703882: 5.0,
 32339963: 2.0,
 31873355: 6.0,
 32912887: 1566.0,
 32905571: 2.0,
 32106292: 2.0,
 31834730: 4393.0,
 32399661: 2.0,
 32396545: 2.0,
 32398228: 2.0,
 32635082: 695.0,
 32873584: 8972.0,
 33309628: 2.0,
 31955854: 33962.0,
 33295823: 218.0,
 32499174: 1271.0,
 32905445: 2.0,
 32905769: 4.0,
 31825343: 5.0,
 32989236: 2.0,
 31857053: 1040.0,
 32040212: 8.0,
 32905551: 4.0,
 32339967: 2.0,
 32126159: 5.0,
 33205192: 1110.0,
 32905589: 4.0,
 32904807: 2.0}

In [16]:
len(questionable_changes)

33

In [17]:
len(set(questionable_changes))

33

#### We checked GPS errors and found multiple questionable changes, meaning that bike was missing from GPS for longer time than just an momentarily variation in location. 
#### Changes in location were controled (making distance controle for that) and conclusion is thata everything with same uid should be merged. Rows with same uid's will be merged and minutes of standing added together + difference in minutes that looks like a ride but it is only GPS error

### Running cells to clean the data from extra rows (temporar solutions for distance check and for choosing lat and lng)

In [18]:
for uid in uidsList: 
    single_uid_records = df[df['uid']==uid]
    first_row_data = single_uid_records.iloc[0]
    last_end_time = first_row_data['time_end']
    current_sum = first_row_data['standing_time_minutes']
    last_oi = first_row_data['old_index']


    new_df = new_df.append({'old_index': first_row_data['old_index'], 'lat' : first_row_data['lat'], 'lng' : first_row_data['lng'], 
                            'time_begin' : first_row_data['time_begin'],
                            'time_end' : last_end_time, 'standing_time_minutes' : first_row_data['standing_time_minutes'],
                            'uid' : first_row_data['uid'], 'name' : first_row_data['name'], 'address' : first_row_data['address'],
                            'bike' : first_row_data['bike'], 'bike_racks' : first_row_data['bike_racks'], 
                            'bikes' : first_row_data['bikes'], 'booked_bikes' : first_row_data['booked_bikes'], 
                            'free_racks' : first_row_data['free_racks'], 'free_special_racks' : first_row_data['free_special_racks'],
                            'maintenance' : first_row_data['maintenance'], 'number' : first_row_data['number'], 
                            'place_type' : first_row_data['place_type'], 'rack_locks' : first_row_data['rack_locks'],
                            'special_racks' : first_row_data['special_racks'], 'spot' : first_row_data['spot'], 
                            'terminal_type' : first_row_data['terminal_type'], 'city' : first_row_data['city'],
                            'country_code' : first_row_data['country_code'], 'company' : first_row_data['company'],
                            'timezone' : first_row_data['timezone'], 'available_bikes' : first_row_data['available_bikes']}, ignore_index=True) 

    for index, row in islice(single_uid_records.iterrows(), 1, None):
        if last_oi+1 != df['old_index'][index]:
            break
        t_begin = df['time_begin'][index].split('.')[0]
        t_end_previous = last_end_time.split('.')[0]    

        d1 = datetime.strptime(t_begin, fmt)
        d2 = datetime.strptime(t_end_previous, fmt)

        d1_ts = time.mktime(d1.timetuple())
        d2_ts = time.mktime(d2.timetuple())

        minsDiff = round((int(d1_ts-d2_ts) / 60), 0)
        current_sum += row['standing_time_minutes'] + minsDiff

        new_df.iloc[-1, new_df.columns.get_loc('standing_time_minutes')] = current_sum
        new_df.iloc[-1, new_df.columns.get_loc('lat')] = row['lat']
        new_df.iloc[-1, new_df.columns.get_loc('lng')] = row['lng']
        new_df.iloc[-1, new_df.columns.get_loc('time_end')] = row['time_end']

        last_end_time = row['time_end']
        last_oi = df['old_index'][index]




KeyboardInterrupt: 

In [19]:
new_df.drop(['old_index'], axis=1,inplace=True)

In [20]:
new_df

,lat,lng,time_begin,time_end,standing_time_minutes,uid,name,address,bike,bike_racks,bikes,booked_bikes,free_racks,free_special_racks,maintenance,number,place_type,rack_locks,special_racks,spot,terminal_type,city,country_code,company,timezone,available_bikes
0,50.944121,7.023142,2020-07-01 00:00:01.668676,2020-07-03 18:42:02.297568,4002.0,31797250,BIKE 22822,NaN,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin,1093
1,50.957413,7.011876,2020-07-03 18:54:01.772866,2020-07-03 18:54:01.772866,0.0,31952802,BIKE 22822,NaN,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin,1136
2,50.938537,6.937958,2020-07-03 22:51:02.841407,2020-07-04 02:01:02.253767,190.0,31966763,BIKE 22822,NaN,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin,1099
3,50.977131,6.963488,2020-07-04 02:25:02.497300,2020-07-04 09:38:02.930782,433.0,31975314,BIKE 22822,NaN,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin,1103
4,50.976539,6.968318,2020-07-04 12:00:02.252010,2020-07-04 13:10:02.358804,70.0,31986071,BIKE 22822,NaN,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin,1109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46500,50.947645,6.964613,2020-07-03 16:28:01.772634,2020-07-03 17:49:02.639065,81.0,31943294,BIKE 21751,NaN,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin,1148
46501,50.920422,6.917579,2020-07-03 18:25:01.900419,2020-07-03 18:34:02.133197,9.0,31950734,BIKE 21751,NaN,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin,1138
46502,50.931636,6.948560,2020-07-03 18:56:02.114210,2020-07-03 22:59:01.927901,243.0,31952985,BIKE 21751,NaN,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin,1134
46503,50.919052,6.920724,2020-07-03 23:18:02.023834,2020-07-04 14:23:02.146589,905.0,31968982,BIKE 21751,NaN,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin,1097


In [21]:
new_df.to_csv('processed_data_nextbike_2020-01cc.csv', sep=';', index=False) 

### Checking the data

In [ ]:
single_uid = df[df['uid']== 26062537 ]
single_uid

In [ ]:
single_uid = new_df[new_df['uid']== 26062537 ]
single_uid

In [ ]:
new_df

In [ ]:
uidsToList2 = new_df['uid'].tolist()
uidsList2 = list(set(uidsToList))

In [ ]:
len(uidsList2)

In [ ]:
len(set(uidsList2))

In [ ]:
new_df['standing_time_minutes'].value_counts(dropna=False)

In [ ]:
quid= df[df['uid']==25690126]
quid